In [0]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7821 sha256=c23a5667d97ee1bed08d3072965365a98ac939b0667eaa0f890bfb50fa8ad791
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [0]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [0]:
product = pd.read_csv('/content/newflipkart1.csv')
product.shape

(19998, 3)

In [0]:
product = product.dropna()
#na_free = product.dropna()
#only_na = product[np.invert(product.index.isin(na_free.index))]
product.shape

(19998, 3)

In [0]:
product1 = product[product.notnull()].copy()
product1["description"].head(10)
product1.shape

(19998, 3)

In [0]:
product1['Key_words'] = ""


for index, row in product1.iterrows():
    description_var = row['description']

    r = Rake()
    r.extract_keywords_from_text(description_var)
    key_words_dict_scores = r.get_word_degrees()

    row['Key_words'] = list(key_words_dict_scores.keys())
    row['Key_words'] = " ".join(row['Key_words'])

# dropping the description column
product1.drop(columns = ['description'], inplace = True)


In [0]:
product1.head(20)

,product_name,pid,Key_words
0,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,lukewarm water 3 fabric cotton lycra type cycl...
1,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,sofas periodically storage included replaced b...
2,AW Bellies,SHOEH4GRSUBJGZXE,499 material use shoe bags clean cloth specifi...
3,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,lukewarm water women specifications bleach add...
4,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,df purpose arnica dog shampoo specifications 1...
5,Eternal Gandhi Super Series Crystal Paper Weig...,PWTEB7H2E4KCYUE3,general model name gandhi paper weight mark v ...
6,Alisha Solid Women's Cycling Shorts,SRTEH2FVVKRBAXHB,lukewarm water alisha solid women specificatio...
7,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGYGHFUEXN,sofas periodically storage included replaced b...
8,"dilli bazaaar Bellies, Corporate Casuals, Casuals",SHOEH3DZBFR88SCK,party dilli bazaaar bellies pink heel height 0...
9,Alisha Solid Women's Cycling Shorts,SRTEH2FVUHAAVH9X,lukewarm water women specifications 4 fabric c...


In [0]:
vectorizer = TfidfVectorizer(stop_words='english')
tfid_matrix = vectorizer.fit_transform(product1["Key_words"])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)

In [0]:
indices = pd.Series(product1["product_name"])

def recommendations(title, cosine_sim = cosine_sim):

    recommended_movies = []

    idx = indices[indices == title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    top_10_indexes = list(score_series.iloc[1:11].index)
    
    recommen_prod_index = []

    # populating the list with the titles of the best 10 matching products
    for i in top_10_indexes:
        recommen_prod_index.append(list(product1.index)[i])

    recommen_prod_names = []

    for i in recommen_prod_index:
      recommen_prod_names.append(product1.iloc[[i],[0]])
      
    return recommen_prod_names 

In [0]:
recommendations('''FabHomeDecor Fabric Double Sofa Bed''')

[                          product_name
 7  FabHomeDecor Fabric Double Sofa Bed,
                            product_name
 16  FabHomeDecor Fabric Double Sofa Bed,
                            product_name
 19  FabHomeDecor Fabric Double Sofa Bed,
                         product_name
 9945  Handiana Solid Wood Dining Set,
                         product_name
 9942  Handiana Solid Wood Dining Set,
                         product_name
 9950  Handiana Solid Wood Dining Set,
                                      product_name
 17875  Stellar Engineered Wood Entertainment Unit,
                                   product_name
 18648  Ethnic Handicrafts Solid Wood Queen Bed,
                                    product_name
 18668  Ethnic Handicrafts Solid Wood Single Bed,
                                   product_name
 19081  @home Annulus Solid Wood Dressing Table]

In [0]:
import pickle
pickle.dump(cosine_similarity, open('content_Filtering_v1.pickle', 'wb'))

In [0]:
import pickle
model = pickle.load(open('content_Filtering_v1.pickle', 'rb'))

In [0]:
result = model([2000][15])